# Calculate circadian scores of genes in extracted hetionet paths

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Read in extracted drug->disease edges 

In [2]:
# function to read in edge files
def read_edge_file(file_name):
    '''
    file_name: edge file name
    '''
    edge_gene_dict = {}
    f = open(file_name, 'r')
    for line in f:
        line = line.strip()
        line_s = line.split(';')
        pair_id = line_s[0]
        # new add new item to the dictionary
        if not (pair_id in edge_gene_dict.keys()):
            drug_hl = line_s[1]
            drug = line_s[4].split('::')[1]
            disease = line_s[-1].split('::')[1]
            edge_gene_dict[pair_id] = {'half_life':float(drug_hl), 'drug':drug, 'disease':disease, 
                                       'G':[], 'GC':[], 'GD':[], 'CG':[], 'DG':[]}
        # add gene to metapath class
        meta_class = line_s[2]
        if meta_class == 'G' or meta_class == 'GC' or meta_class == 'GD':
            gene_id = 5
        else:
            gene_id = 6
        gene = int(line_s[gene_id].split('::')[1])
        edge_gene_dict[pair_id][meta_class].append(gene)
            
    return edge_gene_dict

In [3]:
# circadian drug~disease pairs from Ruben et al data, which will be used as positive standard
rb_edge_dict = read_edge_file('data/ruben_hetionet_edges.txt')
len(rb_edge_dict)

24

In [4]:
# short-half-life drug~disease pairs from DrugBank, which will be used as negative standard
db_edge_dict = read_edge_file('data/drugbank_hetionet_edges.txt')
len(db_edge_dict)

387

### Assign circadian score to drug~disease pair based on genes in the path 

In [5]:
# read in processed CircaDB data 
circa_db = pd.read_csv('data/circa_db_mapped.tsv', sep = '\t')
circa_db.head()

,gene_id,Fat SQ_fdr,Fat Visceral_fdr,Aorta_fdr,Artery Coronary_fdr,Artery Tibial_fdr,Colon_fdr,Esophagus_fdr,Heart Atrial_fdr,Liver_fdr,...,Artery Coronary_exp,Artery Tibial_exp,Colon_exp,Esophagus_exp,Heart Atrial_exp,Liver_exp,Lung_exp,Nerve Tibial_exp,Pituitary_exp,Thyroid_exp
0,653635,0.6432,0.1526,0.8443,0.7712,0.9549,0.5059,0.2928,0.6953,0.9732,...,12.300,11.590,12.7200,12.3033,5.369,5.406,13.680,19.480,15.840,19.2550
1,79854,0.7652,0.2412,0.0473,0.0002,0.6314,0.8602,0.0820,0.6481,0.6405,...,5.940,8.419,5.4407,4.6607,2.962,3.240,7.024,12.110,9.898,9.7615
2,643837,0.9075,0.0774,0.7887,0.2173,0.6017,0.3294,0.0696,0.2758,0.3438,...,9.039,11.520,7.1738,7.6350,14.350,3.674,6.016,5.872,18.600,6.5845
3,26155,0.8656,0.4570,0.5405,0.6391,0.8885,0.3902,0.6801,0.8984,0.3532,...,52.240,62.410,50.8450,62.9550,30.670,28.420,57.320,69.660,57.560,66.5800
4,339451,0.8808,0.6235,0.4360,0.4999,0.7446,0.7780,0.6238,0.6645,0.9089,...,13.510,10.800,11.6200,14.5633,4.870,5.979,15.600,9.224,21.420,15.4450


In [6]:
## extract amplitutde columns 
circa_db_cols = list(circa_db.columns)
circa_db_col_types = [x.split('_')[1] for x in circa_db_cols]
amp_cols = [x == 'amp' for x in circa_db_col_types]
circa_db_amp = circa_db.loc[:,amp_cols]
circa_db_amp.head()

,Fat SQ_amp,Fat Visceral_amp,Aorta_amp,Artery Coronary_amp,Artery Tibial_amp,Colon_amp,Esophagus_amp,Heart Atrial_amp,Liver_amp,Lung_amp,Nerve Tibial_amp,Pituitary_amp,Thyroid_amp
0,0.1103,0.2911,0.1176,0.2648,0.0485,0.1284,0.2597,0.1626,0.0873,0.0688,0.0908,0.0684,0.1909
1,0.1546,0.3117,0.4138,0.7053,0.0967,0.3035,0.4461,0.2465,0.0465,0.0966,0.0404,0.0482,0.4191
2,0.0588,0.3382,0.0259,0.1445,0.0165,0.1099,0.2088,0.1776,0.2813,0.0329,0.0781,0.1048,0.0400
3,0.0872,0.1537,0.0414,0.1185,0.0680,0.0926,0.1006,0.0819,0.1841,0.1675,0.1504,0.0531,0.0790
4,0.1180,0.2041,0.1266,0.0513,0.1107,0.1788,0.1924,0.0503,0.1025,0.0857,0.1918,0.1505,0.3265


In [7]:
# function to calculate mean circadian scores of input genes
def mean_score(query_gene_list, all_genes, gene_scores):
    '''
    query_gene_list: list that contains input genes
    all_gene: one column of dataframe that contains entrez ID of genes
    gene_scores: dataframe that contains circadian scores of genes. Each column represents one tissue
    '''
    if len(query_gene_list) == 0:
        score = float('NaN')
    else:
        gene_count = 0
        score = 0
        unique_genes = np.unique(query_gene_list)
        unique_len = len(unique_genes)
        for g in range(0, unique_len):
            gs = gene_scores[all_genes == unique_genes[g]]
            if len(gs) == 1:
                gene_count = gene_count + 1
                score = score + float(gs)
        if gene_count == 0:
            score = float('NaN')
        else:
            score = score/gene_count
    
    return score


# function to calculate weighted mean circadian scores of input genes
def weighted_mean_score(query_gene_list, all_genes, gene_scores):
    '''
    query_gene_list: list that contains input genes
    all_gene: one column of dataframe that contains entrez ID of genes
    gene_scores: one column of dataframe that contains circadian scores of genes in one tissue
    '''
    if len(query_gene_list) == 0:
        score = float('NaN')
    else:
        gene_count = 0
        score = 0
        unique_genes = np.unique(query_gene_list)
        unique_len = len(unique_genes)
        for g in range(0, unique_len):
            gs = gene_scores[all_genes == unique_genes[g]]
            if len(gs) == 1:
                count = query_gene_list.count(unique_genes[g])
                gene_count = gene_count + count
                score = score + float(gs) * count
        if gene_count == 0:
            score = float('NaN')
        else:
            score = score/gene_count
    
    return score


# function to calculate max circadian scores of input genes
def max_score(query_gene_list, all_genes, gene_scores):
    '''
    query_gene_list: list that contains input genes
    all_gene: one column of dataframe that contains entrez ID of genes
    gene_scores: one column of dataframe that contains circadian scores of genes in one tissue
    '''
    if len(query_gene_list) == 0:
        score = float('NaN')
    else:
        gene_count = 0
        score = 0
        unique_genes = np.unique(query_gene_list)
        unique_len = len(unique_genes)
        for g in range(0, unique_len):
            gs = gene_scores[all_genes == unique_genes[g]]
            if len(gs) == 1:
                gene_count = gene_count + 1
                if float(gs) > score:
                    score = float(gs)
        if gene_count == 0:
            score = float('NaN')
    
    return score


# function to assign circadian scores to each drug~disease pair based on edges extracted from hetionet
def assign_edge_circadian_score(edge_dict, genes, gene_scores, method):
    '''
    edge_dict: dictionary that contains drug name, disease name, half-life, and genes in each drug~disease path
    genes: one column of dataframe that contains entrez ID of genes
    gene_scores: dataframe that contains circadian scores of gene. Each column represents one tissue
    methods: {'mean','wmean','max'}
    '''
    edge_score_dict = {}
    for edge_id, edge_info in edge_dict.items():
        edge_score_dict[edge_id] = {}
        for info_type, info in edge_info.items():
            if info_type == 'half_life' or info_type == 'drug' or info_type == 'disease':
                edge_score_dict[edge_id][info_type] = info
            else:
                for gc in gene_scores.columns:
                    score_name = info_type + '_' + gc
                    if method == 'mean':
                        edge_score_dict[edge_id][score_name] = mean_score(info, genes, gene_scores[gc])
                    if method == 'wmean':
                        edge_score_dict[edge_id][score_name] = weighted_mean_score(info, genes, gene_scores[gc])
                    if method == 'max':
                        edge_score_dict[edge_id][score_name] = max_score(info, genes, gene_scores[gc])                        
                    
    return edge_score_dict   

In [8]:
# Calculate mean circadian score for Ruben et al pairs 
rb_mean_dict = assign_edge_circadian_score(rb_edge_dict, circa_db['gene_id'], circa_db_amp, 'mean')
rb_mean_df = pd.DataFrame.from_dict(rb_mean_dict, orient = 'index')
# Calculate mean circadian score for DrugBank pairs
db_mean_dict = assign_edge_circadian_score(db_edge_dict, circa_db['gene_id'], circa_db_amp, 'mean')
db_mean_df = pd.DataFrame.from_dict(db_mean_dict, orient = 'index')

In [9]:
# Calculate weighted mean circadian score for Ruben et al pairs
rb_wmean_dict = assign_edge_circadian_score(rb_edge_dict, circa_db['gene_id'], circa_db_amp, 'wmean')
rb_wmean_df = pd.DataFrame.from_dict(rb_wmean_dict, orient = 'index')
# Calculate weighted mean circadian score for DrugBank pairs
db_wmean_dict = assign_edge_circadian_score(db_edge_dict, circa_db['gene_id'], circa_db_amp, 'wmean')
db_wmean_df = pd.DataFrame.from_dict(db_wmean_dict, orient = 'index')

In [10]:
# Calculate max circadian score for Ruben et al pairs
rb_max_dict = assign_edge_circadian_score(rb_edge_dict, circa_db['gene_id'], circa_db_amp, 'max')
rb_max_df = pd.DataFrame.from_dict(rb_max_dict, orient = 'index')
# Calculate max circadian score for DrugBank pairs
db_max_dict = assign_edge_circadian_score(db_edge_dict, circa_db['gene_id'], circa_db_amp, 'max')
db_max_df = pd.DataFrame.from_dict(db_max_dict, orient = 'index')

### Combine scores of Ruben et al and Drugbank into one dataframe

In [11]:
# function to combine two dataframes that contain calculated circiadian scores 
def combine_ruben_drugbank(rb_df, db_df):
    '''
    rb_df: dataframe of Ruben et al drug~disease pairs
    db_df: dataframe of Ruben et al drug~disease pairs
    '''
    rb_len = len(rb_df)
    db_len = len(db_df)
    # make column indicating whether drug~disease has circadian efficacy
    rb_status = list(np.repeat('Y', rb_len))
    db_status = list(np.repeat('N', db_len))
    for i in range(0, db_len):
        if db_df.half_life.iloc[i,] < 12:
            db_status[i] = db_status[i] + ',HL<12h'
        else:
            db_status[i] = db_status[i] + ',12h<HL<24h'
    circa_status = np.concatenate((rb_status, db_status), axis=0)
    # combine two data frames, add the new column
    combine_df = pd.concat([rb_df,db_df])
    combine_df.insert(3,'circadian_efficacy',circa_status)
    
    return combine_df

In [12]:
# combine mean scores of Ruben et al and Drugbank
combine_mean_df = combine_ruben_drugbank(rb_mean_df,db_mean_df)
combine_mean_df.to_csv('data/hetionet_edges_mean_circadian_score.tsv', sep = '\t', na_rep = 'NA', 
                       float_format = '%.4f', index = False)
combine_mean_df.head()

,half_life,drug,disease,circadian_efficacy,G_Fat SQ_amp,G_Fat Visceral_amp,G_Aorta_amp,G_Artery Coronary_amp,G_Artery Tibial_amp,G_Colon_amp,...,DG_Artery Coronary_amp,DG_Artery Tibial_amp,DG_Colon_amp,DG_Esophagus_amp,DG_Heart Atrial_amp,DG_Liver_amp,DG_Lung_amp,DG_Nerve Tibial_amp,DG_Pituitary_amp,DG_Thyroid_amp
1,2.5,DB00635,DOID:2841,Y,0.170200,0.388300,0.190000,0.365100,0.175200,0.207200,...,0.338625,0.323501,0.261203,0.303874,0.349518,0.260909,0.226819,0.175107,0.243941,0.256928
10,2.5,DB00635,DOID:7148,Y,0.208000,0.293900,0.166300,0.206850,0.217350,0.128800,...,0.277648,0.231370,0.202427,0.268224,0.264927,0.217534,0.180166,0.149104,0.188192,0.215489
11,3.0,DB00881,DOID:10763,Y,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,35.0,DB00584,DOID:10763,Y,0.328667,0.237133,0.262267,0.431133,0.338033,0.447567,...,0.354138,0.312244,0.269208,0.309841,0.321052,0.268699,0.244218,0.190392,0.237238,0.251429
13,12.0,DB01054,DOID:10763,Y,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# combine weighted  mean scores of Ruben et al and Drugbank
combine_wmean_df = combine_ruben_drugbank(rb_wmean_df,db_wmean_df)
combine_wmean_df.to_csv('data/hetionet_edges_weightedmean_circadian_score.tsv', sep = '\t', na_rep = 'NA', 
                        float_format = '%.4f', index = False)
combine_wmean_df.head()

,half_life,drug,disease,circadian_efficacy,G_Fat SQ_amp,G_Fat Visceral_amp,G_Aorta_amp,G_Artery Coronary_amp,G_Artery Tibial_amp,G_Colon_amp,...,DG_Artery Coronary_amp,DG_Artery Tibial_amp,DG_Colon_amp,DG_Esophagus_amp,DG_Heart Atrial_amp,DG_Liver_amp,DG_Lung_amp,DG_Nerve Tibial_amp,DG_Pituitary_amp,DG_Thyroid_amp
1,2.5,DB00635,DOID:2841,Y,0.170200,0.388300,0.190000,0.365100,0.175200,0.207200,...,0.346277,0.367972,0.288923,0.302225,0.370428,0.279770,0.212391,0.183715,0.254443,0.274076
10,2.5,DB00635,DOID:7148,Y,0.208000,0.293900,0.166300,0.206850,0.217350,0.128800,...,0.310588,0.278687,0.235347,0.289625,0.305955,0.244695,0.197140,0.163254,0.216747,0.243822
11,3.0,DB00881,DOID:10763,Y,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,35.0,DB00584,DOID:10763,Y,0.328667,0.237133,0.262267,0.431133,0.338033,0.447567,...,0.354138,0.312244,0.269208,0.309841,0.321052,0.268699,0.244218,0.190392,0.237238,0.251429
13,12.0,DB01054,DOID:10763,Y,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# combine max scores of Ruben et al and Drugbank
combine_max_df = combine_ruben_drugbank(rb_max_df,db_max_df)
combine_max_df.to_csv('data/hetionet_edges_max_circadian_score.tsv', sep = '\t', na_rep = 'NA', 
                      float_format = '%.4f', index = False)
combine_max_df.head()

,half_life,drug,disease,circadian_efficacy,G_Fat SQ_amp,G_Fat Visceral_amp,G_Aorta_amp,G_Artery Coronary_amp,G_Artery Tibial_amp,G_Colon_amp,...,DG_Artery Coronary_amp,DG_Artery Tibial_amp,DG_Colon_amp,DG_Esophagus_amp,DG_Heart Atrial_amp,DG_Liver_amp,DG_Lung_amp,DG_Nerve Tibial_amp,DG_Pituitary_amp,DG_Thyroid_amp
1,2.5,DB00635,DOID:2841,Y,0.1702,0.3883,0.1900,0.3651,0.1752,0.2072,...,1.4109,2.8025,1.0454,0.9365,1.2699,0.7149,1.1211,0.8178,1.0484,2.7498
10,2.5,DB00635,DOID:7148,Y,0.2458,0.3883,0.1900,0.3651,0.2595,0.2072,...,1.8474,2.8025,0.8333,1.1033,1.0130,1.1124,1.2282,1.1831,1.1350,2.7498
11,3.0,DB00881,DOID:10763,Y,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,35.0,DB00584,DOID:10763,Y,0.6107,0.4918,0.6327,0.7085,0.5969,0.5796,...,1.8096,2.8025,1.0454,0.9365,1.2699,0.8190,1.1325,0.8261,1.0484,2.7498
13,12.0,DB01054,DOID:10763,Y,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
